In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import datetime

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def get_hours(iso, peak_type, period): 
  list_iso = ['PJM', 'MISO', 'ERCOT', 'SPP', 'NYISO', 'WECC', 'CAISO']
  list_peak_type = ['onpeak', 'offpeak', 'flat', '2x16H', '7x8']
  list_quarter = ['Q1', 'Q2', 'Q3', 'Q4']
  list_month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

  if not iso in list_iso: 
    return 'Wrong ISO'

  elif not peak_type in list_peak_type: 
    return 'Wrong Peak Type'
    
  else: 
    # Get year; assume year range is 1000 to 9999
    year = period[:4]
    try: 
      int(year)
    except: 
      return 'Wrong Time Period'
    year = int(year)
    
    # Get period type
    time_type = period[4:]
    # Annually
    if time_type == 'A': 
      return get_hours_annually(iso, peak_type, year)
    # Quarterly
    elif time_type in list_quarter: 
      return get_hours_quarterly(iso, peak_type, year, list_quarter.index(time_type)+1)
    # Monthly
    elif time_type in list_month: 
      return get_hours_monthly(iso, peak_type, year, list_month.index(time_type)+1)
    # Daily
    else: 
      date = time_type.split('-')
      if not len(date) == 3: 
        return 'Wrong Time Period'
      else: 
        try: 
          int(date[1]) + int(date[2])
        except:
          return 'Wrong Time Period'
        month = int(date[1])
        day = int(date[2])
        try: 
          datetime.date(year, month, day)
        except:
          return 'Wrong Time Period'
        return get_hours_daily(iso, peak_type, datetime.date(year, month, day))

In [ ]:
def get_hours_from_type(peak_type, num_workday, num_restday, daylight_saving): 
  if peak_type == 'onpeak': 
    return num_workday * 16
  elif peak_type == 'offpeak': 
    return num_workday * 8 + num_restday * 24 + daylight_saving
  elif peak_type == 'flat': 
    return (num_workday + num_restday) * 24 + daylight_saving
  elif peak_type == '2x16H': 
    return num_restday * 16
  elif peak_type == '7x8': 
    return (num_workday + num_restday) * 8 + daylight_saving
  else: 
    # Should not occur
    return 'Wrong Attempt get_hours_from_type'
    exit()

In [ ]:
def is_leap_year(year): 
  if year % 4 == 0 and not year % 100 == 0: 
    return True
  elif year % 400 == 0: 
    return True
  else:
    return False

def is_NERC_holiday(date): 
  # New Year's Day
  if date.month == 1 and date.day == 1 and not date.isoweekday() == 7: 
    return True
  elif date.month == 1 and date.day == 2 and date.isoweekday() == 1: 
    return True
  # Independence Day
  elif date.month == 7 and date.day == 4 and not date.isoweekday() == 7: 
    return True
  elif date.month == 7 and date.day == 5 and date.isoweekday() == 1: 
    return True
  # Christmas Day
  elif date.month == 12 and date.day == 25 and not date.isoweekday() == 7: 
    return True
  elif date.month == 12 and date.day == 26 and date.isoweekday() == 1: 
    return True
  # Memorial Day
  elif date.month == 5 and date.day in range(25, 32) and date.isoweekday() == 1: 
    return True
  # Labor Day
  elif date.month == 9 and date.day in range(1, 8) and date.isoweekday() == 1: 
    return True
  # Thanksgiving Day
  elif date.month == 11 and date.day in range(22, 29) and date.isoweekday() == 4: 
    return True
  
  else:
    return False

def region_iso(iso): 
  if iso in ['WECC', 'CAISO']: 
    return 'western'
  elif iso in ['PJM', 'MISO', 'ERCOT', 'SPP', 'NYISO']: 
    return 'eastern'
  else: 
    return 'Wrong ISO'

In [ ]:
def get_hours_annually(iso, peak_type, year): 
  region = region_iso(iso)
  
  # Western market
  if region == 'western': 
    num_restday = 52
    num_workday = 52 * 6
    # Adjust for one more day
    if datetime.date(year, 1, 1).isoweekday() in [7]: 
      num_restday += 1
    else: 
      num_workday += 1
    # Adjust for leap year
    if is_leap_year(year): 
      if datetime.date(year, 1, 2).isoweekday() in [7]: 
        num_restday += 1
      else: 
        num_workday += 1
    # Adjust for holidays
    num_restday += 6
    num_workday -= 6
  
  # Eastern market
  elif region == 'eastern': 
    num_restday = 52 * 2
    num_workday = 52 * 5
    # Adjust for one more day
    if datetime.date(year, 1, 1).isoweekday() in [6, 7]: 
      num_restday += 1
    else: 
      num_workday += 1
    # Adjust for leap year
    if is_leap_year(year): 
      if datetime.date(year, 1, 2).isoweekday() in [6, 7]: 
        num_restday += 1
      else: 
        num_workday += 1
    # Adjust for Memorial Day, Labor Day, Thanksgiving Day
    num_restday += 3
    num_workday -= 3
    # Adjust for New Year's Day
    if not datetime.date(year, 1, 1).isoweekday() == 6: 
      num_restday += 1
      num_workday -= 1
    # Adjust for Independence Day
    if not datetime.date(year, 7, 4).isoweekday() == 6: 
      num_restday += 1
      num_workday -= 1
    # Adjust for Christmas Day
    if not datetime.date(year, 12, 25).isoweekday() == 6: 
      num_restday += 1
      num_workday -= 1
  
  # Unknown market
  else: 
    # Should not occur
    return 'Wrong Attempt get_hours_annually'
    exit()
  

  daylight_saving = 0
  
  return get_hours_from_type(peak_type, num_workday, num_restday, daylight_saving)

In [ ]:
def get_hours_quarterly(iso, peak_type, year, quarter): 
  region = region_iso(iso)
  
  # Western market
  if region == 'western': 
    num_restday = 13
    num_workday = 13 * 6
    
    # Adjust for Q1
    if quarter == 1: 
      # Adjust for leap year and one less day
      if not is_leap_year(year): 
        if datetime.date(year, 4, 1).isoweekday() in [7]: 
          num_restday -= 1
        else: 
          num_workday -= 1
      # Adjust for New Year's Day
      num_restday += 1
      num_workday -= 1
    
    # Adjust for Q2
    elif quarter == 2: 
      # Adjust for Memorial Day
      num_restday += 1
      num_workday -= 1
    
    # Adjust for Q3
    elif quarter == 3: 
      # Adjust for one more day
      if datetime.date(year, 7, 1).isoweekday() in [7]: 
        num_restday += 1
      else: 
        num_workday += 1
      # Adjust for Independence Day, Labor Day
      num_restday += 2
      num_workday -= 2
    
    # Adjust for Q4
    elif quarter == 4: 
      # Adjust for one more day
      if datetime.date(year, 10, 1).isoweekday() in [7]: 
        num_restday += 1
      else: 
        num_workday += 1
      # Adjust for Thanksgiving Day, Christmas Day
      num_restday += 2
      num_workday -= 2
    
    # Unknown quarter
    else: 
      # Should not occur
      return 'Wrong Attempt get_hours_quarterly A'
      exit()
  
  # Eastern market
  elif region == 'eastern': 
    num_restday = 13 * 2
    num_workday = 13 * 5
    
    # Adjust for Q1
    if quarter == 1: 
      # Adjust for leap year and one less day
      if not is_leap_year(year): 
        if datetime.date(year, 4, 1).isoweekday() in [6, 7]: 
          num_restday -= 1
        else: 
          num_workday -= 1
      # Adjust for New Year's Day
      if not datetime.date(year, 1, 1).isoweekday() == 6: 
        num_restday += 1
        num_workday -= 1
    
    # Adjust for Q2
    elif quarter == 2: 
      # Adjust for Memorial Day
      num_restday += 1
      num_workday -= 1
    
    # Adjust for Q3
    elif quarter == 3: 
      # Adjust for one more day
      if datetime.date(year, 7, 1).isoweekday() in [6, 7]: 
        num_restday += 1
      else: 
        num_workday += 1
      # Adjust for Independence Day
      if not datetime.date(year, 7, 4).isoweekday() == 6: 
        num_restday += 1
        num_workday -= 1
      # Adjust for Labor Day
      num_restday += 1
      num_workday -= 1
    
    # Adjust for Q4
    elif quarter == 4: 
      # Adjust for one more day
      if datetime.date(year, 10, 1).isoweekday() in [6, 7]: 
        num_restday += 1
      else: 
        num_workday += 1
      # Adjust for Christmas Day
      if not datetime.date(year, 12, 25).isoweekday() == 6: 
        num_restday += 1
        num_workday -= 1
      # Adjust for Thanksgiving Day
      num_restday += 1
      num_workday -= 1
    
    # Unknown quarter
    else: 
      # Should not occur
      return 'Wrong Attempt get_hours_quarterly B'
      exit()
  
  # Unknown market
  else: 
    # Should not occur
    return 'Wrong Attempt'
    exit()



  # Adjust for daylight-saving setting
  if iso == 'MISO' or quarter in [2, 3]: 
    daylight_saving = 0
  elif quarter == 1: 
    daylight_saving = -1
  elif quarter == 4: 
    daylight_saving = 1
  # Unknown quarter
  else: 
    # Should not occur
    return 'Wrong Attempt'
    exit()

  return get_hours_from_type(peak_type, num_workday, num_restday, daylight_saving)

In [ ]:
def get_hours_monthly(iso, peak_type, year, month): 
  region = region_iso(iso)
  
  # Western market
  if region == 'western': 
    num_restday = 4
    num_workday = 4 * 6
    
    # Adjust for M2
    if month == 2: 
      # Adjust for leap year
      if is_leap_year(year): 
        if datetime.date(year, month, 1).isoweekday() in [7]: 
          num_restday += 1
        else: 
          num_workday += 1
    # Adjust for two more days in other months
    else: 
      if datetime.date(year, month, 1).isoweekday() in [7]: 
        num_restday += 1
      else: 
        num_workday += 1
      if datetime.date(year, month, 2).isoweekday() in [7]: 
        num_restday += 1
      else: 
        num_workday += 1
    # Adjust for one more day in M1, M3, M5, M7, M8, M10, M12
    if month in [1, 3, 5, 7, 8, 10, 12]: 
      if datetime.date(year, month, 3).isoweekday() in [7]: 
        num_restday += 1
      else: 
        num_workday += 1
    # Adjust for holidays
    if month in [1, 5, 7, 9, 11, 12]: 
      num_restday += 1
      num_workday -= 1
  
  # Eastern market
  elif region == 'eastern': 
    num_restday = 4 * 2
    num_workday = 4 * 5
    
    # Adjust for M2
    if month == 2: 
      # Adjust for leap year
      if is_leap_year(year): 
        if datetime.date(year, month, 1).isoweekday() in [6, 7]: 
          num_restday += 1
        else: 
          num_workday += 1
    # Adjust for two more days in other months
    else: 
      if datetime.date(year, month, 1).isoweekday() in [6, 7]: 
        num_restday += 1
      else: 
        num_workday += 1
      if datetime.date(year, month, 2).isoweekday() in [6, 7]: 
        num_restday += 1
      else: 
        num_workday += 1
    # Adjust for one more day in M1, M3, M5, M7, M8, M10, M12
    if month in [1, 3, 5, 7, 8, 10, 12]: 
      if datetime.date(year, month, 3).isoweekday() in [6, 7]: 
        num_restday += 1
      else: 
        num_workday += 1
    # Adjust for Memorial Day, Labor Day, Thanksgiving Day
    if month in [5, 9, 11]: 
      num_restday += 1
      num_workday -= 1
    # Adjust for New Year's Day
    if month == 1 and not datetime.date(year, 1, 1).isoweekday() == 6: 
      num_restday += 1
      num_workday -= 1
    # Adjust for Independence Day
    if month == 7 and not datetime.date(year, 7, 4).isoweekday() == 6: 
      num_restday += 1
      num_workday -= 1
    # Adjust for Christmas Day
    if month == 12 and not datetime.date(year, 12, 25).isoweekday() == 6: 
      num_restday += 1
      num_workday -= 1
  
  # Unknown market
  else: 
    # Should not occur
    return 'Wrong Attempt get_hours_monthly A'
    exit()



  # Adjust for daylight-saving setting
  if iso == 'MISO' or not month in [3, 11]: 
    daylight_saving = 0
  elif month == 3: 
    daylight_saving = -1
  elif month == 11: 
    daylight_saving = 1
  # Unknown month
  else: 
    # Should not occur
    return 'Wrong Attempt get_hours_monthly B'
    exit()

  return get_hours_from_type(peak_type, num_workday, num_restday, daylight_saving)

In [ ]:
def get_hours_daily(iso, peak_type, date): 
  region = region_iso(iso)
  
  # Western market
  if region == 'western': 
    num_restday = 0
    num_workday = 0
    if date.isoweekday() in [7] or is_NERC_holiday(date): 
      num_restday += 1
    else: 
      num_workday += 1
  
  # Eastern market
  elif region == 'eastern': 
    num_restday = 0
    num_workday = 0
    if date.isoweekday() in [6, 7] or is_NERC_holiday(date): 
      num_restday += 1
    else: 
      num_workday += 1
  
  # Unknown market
  else: 
    # Should not occur
    return 'Wrong Attempt get_hours_daily'
    exit()



  # Adjust for daylight-saving setting
  if iso == 'MISO': 
    daylight_saving = 0
  else: 
    if date.month == 3 and date.day in range(8, 15) and date.isoweekday() == 7: 
      daylight_saving = -1
    elif date.month == 11 and date.day in range(1, 7) and date.isoweekday() == 7: 
      daylight_saving = 1
    else: 
      daylight_saving = 0

  return get_hours_from_type(peak_type, num_workday, num_restday, daylight_saving)

In [ ]:
# Test if the function works
list_peak_type = ['onpeak', 'offpeak', 'flat', '2x16H', '7x8']
for period in ['2018A', '2018Q2', '2018Mar', '2018-2-3']: 
  iso = 'PJM'
  for peak_type in list_peak_type: 
    print(iso, peak_type, period)
    print('hours: ', get_hours(iso, peak_type, period))

PJM onpeak 2018A
hours:  4080
PJM offpeak 2018A
hours:  4680
PJM flat 2018A
hours:  8760
PJM 2x16H 2018A
hours:  1760
PJM 7x8 2018A
hours:  2920
PJM onpeak 2018Q2
hours:  1024
PJM offpeak 2018Q2
hours:  1160
PJM flat 2018Q2
hours:  2184
PJM 2x16H 2018Q2
hours:  432
PJM 7x8 2018Q2
hours:  728
PJM onpeak 2018Mar
hours:  352
PJM offpeak 2018Mar
hours:  391
PJM flat 2018Mar
hours:  743
PJM 2x16H 2018Mar
hours:  144
PJM 7x8 2018Mar
hours:  247
PJM onpeak 2018-2-3
hours:  0
PJM offpeak 2018-2-3
hours:  24
PJM flat 2018-2-3
hours:  24
PJM 2x16H 2018-2-3
hours:  16
PJM 7x8 2018-2-3
hours:  8


In [ ]:
# Test with EnergyGPS data
list_month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for i in range(12): 
  period = '2023' + list_month[i]
  iso = 'CAISO'
  for peak_type in ['flat', 'onpeak', 'offpeak']: 
    print(iso, peak_type, period)
    print('hours: ', get_hours(iso, peak_type, period))

CAISO flat 2023Jan
hours:  744
CAISO onpeak 2023Jan
hours:  400
CAISO offpeak 2023Jan
hours:  344
CAISO flat 2023Feb
hours:  672
CAISO onpeak 2023Feb
hours:  384
CAISO offpeak 2023Feb
hours:  288
CAISO flat 2023Mar
hours:  743
CAISO onpeak 2023Mar
hours:  432
CAISO offpeak 2023Mar
hours:  311
CAISO flat 2023Apr
hours:  720
CAISO onpeak 2023Apr
hours:  400
CAISO offpeak 2023Apr
hours:  320
CAISO flat 2023May
hours:  744
CAISO onpeak 2023May
hours:  416
CAISO offpeak 2023May
hours:  328
CAISO flat 2023Jun
hours:  720
CAISO onpeak 2023Jun
hours:  416
CAISO offpeak 2023Jun
hours:  304
CAISO flat 2023Jul
hours:  744
CAISO onpeak 2023Jul
hours:  400
CAISO offpeak 2023Jul
hours:  344
CAISO flat 2023Aug
hours:  744
CAISO onpeak 2023Aug
hours:  432
CAISO offpeak 2023Aug
hours:  312
CAISO flat 2023Sep
hours:  720
CAISO onpeak 2023Sep
hours:  400
CAISO offpeak 2023Sep
hours:  320
CAISO flat 2023Oct
hours:  744
CAISO onpeak 2023Oct
hours:  416
CAISO offpeak 2023Oct
hours:  328
CAISO flat 2023Nov
h

In [ ]:
# Test consistency of results
list_quarter = ['Q1', 'Q2', 'Q3', 'Q4']
list_month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
iso = 'ERCOT'
peak_type = 'offpeak'

period = '2020A'
print(iso, peak_type, period)
print('annually hours: ', get_hours(iso, peak_type, period))
sum = 0
for i in range(4): 
  period = '2020' + list_quarter[i]
  sum += get_hours(iso, peak_type, period)
print('total quarterly hours: ', sum)

for i in range(4): 
  period = '2020' + list_quarter[i]
  print(iso, peak_type, period)
  print('quarterly hours: ', get_hours(iso, peak_type, period))
  sum = 0
  for j in range(i*3, i*3+3): 
    period = '2020' + list_month[j]
    sum += get_hours(iso, peak_type, period)
  print('total monthly hours: ', sum)

list_month_daynumber = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
for i in range(12): 
  period = '2020' + list_month[i]
  print(iso, peak_type, period)
  print('monthly hours: ', get_hours(iso, peak_type, period))
  sum = 0
  for j in range(1, list_month_daynumber[i]+1): 
    period = '2020-' + str(i+1) + '-' + str(j)
    sum += get_hours(iso, peak_type, period)
  print('total daily hours: ', sum)

ERCOT offpeak 2020A
annually hours:  4672
total quarterly hours:  4672
ERCOT offpeak 2020Q1
quarterly hours:  1159
total monthly hours:  1159
ERCOT offpeak 2020Q2
quarterly hours:  1160
total monthly hours:  1160
ERCOT offpeak 2020Q3
quarterly hours:  1168
total monthly hours:  1168
ERCOT offpeak 2020Q4
quarterly hours:  1185
total monthly hours:  1185
ERCOT offpeak 2020Jan
monthly hours:  392
total daily hours:  392
ERCOT offpeak 2020Feb
monthly hours:  376
total daily hours:  376
ERCOT offpeak 2020Mar
monthly hours:  391
total daily hours:  391
ERCOT offpeak 2020Apr
monthly hours:  368
total daily hours:  368
ERCOT offpeak 2020May
monthly hours:  424
total daily hours:  424
ERCOT offpeak 2020Jun
monthly hours:  368
total daily hours:  368
ERCOT offpeak 2020Jul
monthly hours:  376
total daily hours:  376
ERCOT offpeak 2020Aug
monthly hours:  408
total daily hours:  408
ERCOT offpeak 2020Sep
monthly hours:  384
total daily hours:  384
ERCOT offpeak 2020Oct
monthly hours:  392
total dai